In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(font_scale=.8, palette=sns.color_palette("Set2", n_colors=2));

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score

# Kategorisk data
Maskininlärningsmodeller kräver generellt att datan de tränas på är numerisk. Men vad gör vi då om vi har följande (enkla) dataset?

In [4]:
customer_satisfaction = pd.DataFrame({
    "Customer_ID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "Satisfaction_Level": ["Low", "Medium", "High", "Low", "High", "Medium", "Medium", "Low", "High", "Medium"]
})
customer_satisfaction

,Customer_ID,Satisfaction_Level
0,1,Low
1,2,Medium
2,3,High
3,4,Low
4,5,High
5,6,Medium
6,7,Medium
7,8,Low
8,9,High
9,10,Medium


`Satisfaction_Level`-kolumnen är kategorisk (och består av textsträngar). Den behöver konverteras till numeriska värden för att vår modell ska kunna förstå den.

I det här exemplet har kategorierna en ordning - `Low` < `Medium` < `High`. Det här är ett exempel på *ordinaldata*.


## Ordinaldata

`sklearn` har en `OrdinalEncoder`-*transformer* som transformerar kategorisk ordinaldata till numeriska värden.

Vi kan ange `categories`-argumentet för att definiera ordningen, från `Low` till `High`. Annars hade de ordnats i alfabetisk ordning.

In [5]:
ordinal_encoder = OrdinalEncoder(categories=[["Low", "Medium", "High"]])
ordinal_encoder.fit_transform(customer_satisfaction[["Satisfaction_Level"]])

array([[0.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [1.],
       [0.],
       [2.],
       [1.]])

`OrdinalEncoder` transformerar kategorierna till numeriska värden. 

Vi kan komma åt kategorierna genom `categories_`-attributet.

In [6]:
ordinal_encoder.categories_

[array(['Low', 'Medium', 'High'], dtype=object)]

I praktiken lär sig `OrdinalEncoder`-transformern vilka kategorier som finns i datan. Vi kan använda `inverse_transform`-metoden för att få fram kategorierna igen från numeriska data.

In [7]:
ordinal_encoder.inverse_transform(np.array([[1], [0], [2]]))

array([['Medium'],
       ['Low'],
       ['High']], dtype=object)

Det är viktigt att förstå vad det här innebär ur maskininlärningssynpunkt. Eftersom våra numeriska värden ökar (från `0` till `2` i det här fallet) innebär det också att en modell kommer att ge observationer med höga värden i `Satisfaction_Level`-kolumnen mer tyngd. Det är precis vad vi önskar när vi har ordinaldata.

## One-hot encoding

I andra fall är det inte alls vad vi önskar. Vi kollar på ett annat enkelt dataset:

In [8]:
# One-Hot Encoding: Car Brands
cars = pd.DataFrame({
    "Car_Brand": ["Toyota", "Ford", "Honda", "BMW", "Toyota", "Ford", "Honda", "BMW"]
})
cars


,Car_Brand
0,Toyota
1,Ford
2,Honda
3,BMW
4,Toyota
5,Ford
6,Honda
7,BMW


Här finns det ingen ordning mellan kategorierna. Om vi använde en `OrdinalEncoder` skulle vissa bilmärken få större vikt i modellen än andra, vilket vi högst sannolikt inte vill.

Här kan vi istället använda *one-hot encoding*. `sklearn` har en annan *transformer* som heter `OneHotEncoder`. Vi använder den på `cars`-datan.

`sparse_output`parametern är satt till `False`. Det gör att vi får en vanlig numpy-*array* istället för en *sparse array*. ^[[Vad är en *sparse array*?](https://docs.scipy.org/doc/scipy/tutorial/sparse.html)]

In [9]:
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoder.fit_transform(cars[["Car_Brand"]])

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]])

Vad hände precis? För att lättare förstå *one-hot encoding*, kan vi använda funktionen `get_dummies()` från pandas. Den gör samma sak, men låter oss titta på resultatet på ett sätt som är lite enklare att förstå.

För att göra jämförelsen ännu tydligare, visar vi de booleska värdena `False` och `True` som nollor och ettor istället genom att sätta `dtype`-parametern till `int`.

Vi stoppar också tillbaka `Car_Brand`-kolumnen för att kunna ha koll på vilket bilmärke de olika raderna representerar. Det skulle vi alltså inte göra om vi vill träna en modell på datan.

In [10]:
onehot_cars = pd.get_dummies(cars, columns=['Car_Brand'], dtype=int)
onehot_cars["Car_Brand"] = cars["Car_Brand"]
onehot_cars

,Car_Brand_BMW,Car_Brand_Ford,Car_Brand_Honda,Car_Brand_Toyota,Car_Brand
0,0,0,0,1,Toyota
1,0,1,0,0,Ford
2,0,0,1,0,Honda
3,1,0,0,0,BMW
4,0,0,0,1,Toyota
5,0,1,0,0,Ford
6,0,0,1,0,Honda
7,1,0,0,0,BMW


*One-hot encoding* skapar en ny kolumn för varje kategori och fyller alla kolumner med nollor, utom kolumnen som representerar värdet i raden.

I den första raden har vi en Toyota. Alla kolumner utom `Car_Brand_Toyota` fylls med nollor.

Jämför outputen från cellen ovan med outputen från `OneHotEncoder`-cellen.

`OneHotEncoder`-*transformern* sparar också kategorierna i `categories_`-attributet. Notera att de ligger i bokstavsordning.

In [11]:
onehot_encoder.categories_

[array(['BMW', 'Ford', 'Honda', 'Toyota'], dtype=object)]

Till skillnad från en `OrdinalEncoder` får alltså alla kategorier samma vikt - inget värde är större än de andra.

## Dummy encoding

Om vi tänker efter lite kanske det slår oss att vi inte behöver alla kolumnerna i en *one-hot encoding*. Om vi tar bort en av kolumnerna kan vi fortfarande identifiera dess rader eftersom de är de enda som kommer att innehålla endast nollor.

Vi testar! `get_dummies()` har en parameter som heter `drop_first`. Om vi sätter den till `True` kommer den första kolumnen (`Car_Brand_BMW` i vårt exempel) tas bort.

In [12]:
dummy_cars = pd.get_dummies(cars, columns=['Car_Brand'], drop_first=True, dtype=int)
dummy_cars["Car_Brand"] = cars["Car_Brand"]
dummy_cars

,Car_Brand_Ford,Car_Brand_Honda,Car_Brand_Toyota,Car_Brand
0,0,0,1,Toyota
1,1,0,0,Ford
2,0,1,0,Honda
3,0,0,0,BMW
4,0,0,1,Toyota
5,1,0,0,Ford
6,0,1,0,Honda
7,0,0,0,BMW


BMW-raderna har nu bara nollor.

`OneHotEncoder`-transformern har en `drop`-parameter. Om vi sätter den till `"first"` gör den samma sak som i exemplet ovan.

In [13]:
dummy_encoder = OneHotEncoder(sparse_output=False, drop="first")
dummy_encoder.fit_transform(cars[["Car_Brand"]])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

Kategorierna är, som vanligt, tillgängliga i `categories_`-attributet. Notera att BMW-kategorin finns kvar - det är bara kolumnen som tagits bort.

In [14]:
dummy_encoder.categories_

[array(['BMW', 'Ford', 'Honda', 'Toyota'], dtype=object)]

Varför ska man göra *dummy encoding* istället? Vissa modeller, till exempel linjära regression-modeller, har av matematiska skäl svårt att hantera *one-hot*-kodade kategorier.

Läs mer om one hot- och dummy encoding i [den här artikeln](https://builtin.com/articles/one-hot-encoding).

### scikit-learn eller pandas?

Så, vilken ska man använda, pandas eller scikit-learn? Svaret är som vanligt: det beror på.

Om datan redan är en pandas `DataFrame` och man håller på och förbereder datan för EDA och analys finns det egentligen ingen större poäng att slänga in scikit-learn för att hantera kategoriska variabler.

När det är dags att förbehandla datan för träning är det dock bättre att använda scikit-learn. En stor fördel med scikit-learns *encoders* är att de fungerar utmärkt som steg i en `Pipeline`, vilket är smidigt när man automatiserar en tränings- och evalueringsprocess.